# Back to the Basic (Part 2-2. Linear Regression)

* 2023.05.01.(Mon)
* Dept. of Math., Inha Univ.
* Byung Chun Kim (wizardbc@gmail.com)

## Linear Model

In [ ]:
from typing import Iterable

class Module:
  def __init__(self):
    raise NotImplementedError

  def set_params(self) -> None:
    raise NotImplementedError
  
  def get_params(self) -> dict:
    raise NotImplementedError
  
  def f(self, x):
    raise NotImplementedError
  
  def __call__(self, x:Iterable) -> tuple:
    return tuple([self.f(t) for t in x])


class LinearModel(Module):
  def __init__(self, weight:float=.0, bias:float=.0) -> None:
    self.set_params(weight, bias)

  def set_params(self, weight:float, bias:float) -> None:
    self.weight = weight
    self.bias = bias

  def get_params(self) -> dict[str,float]:
    return {'weight': self.weight, 'bias':self.bias}

  def f(self, x:float) -> float:
    params = self.get_params()
    w = params.get('weight')
    b = params.get('bias')
    return w * x + b

In [ ]:
x = [1,2,3,4,5]

lin = LinearModel(2.0, 1.0)
lin(x)

## Mean Squared Error and its Gradient

$f_\theta(x) = x w + b$ where $\theta=(w,b)$.

$\operatorname{MSE}_\theta(x, y) = \frac{1}{n}\sum^n_{i=1}(f(x_i) - y_i)^2 = \frac{1}{n}\sum^n_{i=1}(x_iw+b - y_i)^2$.

$$\frac{\partial}{\partial w} \operatorname{MSE}_{\theta}(x,y) = \frac{2}{n}\sum^n_{i=1}x_i(x_iw+b - y_i).$$

$$\frac{\partial}{\partial b} \operatorname{MSE}_{\theta}(x,y) = \frac{2}{n}\sum^n_{i=1}(x_iw+b - y_i).$$

In [ ]:
def mse(model:Module, x:Iterable[float], y_true:Iterable[float]) -> float:
  assert len(x) == len(y_true)
  n = len(x)
  y_pred = model(x)
  return sum([(t-p)**2 for t,p in zip(y_true, y_pred)])/n

def d_mse(model:Module, x:Iterable[float], y_true:Iterable[float]) -> dict[str,float]:
  assert len(x) == len(y_true)
  n = len(x)
  y_pred = model(x)
  dw = sum([x*(p-t) for x,p,t in zip(x, y_pred, y_true)])*2/n
  db = sum([(p-t) for p,t in zip(y_pred, y_true)])*2/n
  return {'d_weight': dw, 'd_bias':db}

In [ ]:
mse(lin, [0.0], [2.0])

In [ ]:
d_mse(lin, [0.0], [2.0])

## Gradient Descent

* Update $\theta$: $$\theta_{\textrm{new}} = \theta_{\textrm{old}} - \alpha \nabla\operatorname{MSE}_{\theta_{\textrm{old}}}(x,y)$$ where $\alpha>0$ is a learning rate.

In [ ]:
def update(model:Module, lr:float, d_weight:float, d_bias:float) -> None:
  params_old = model.get_params()
  params_new = {
    'weight': params_old.get('weight') - lr*d_weight,
    'bias': params_old.get('bias') - lr*d_bias
  }
  model.set_params(**params_new)

## Data

In [ ]:
import random
import matplotlib.pyplot as plt

# dataset

# y = a*x + b
f = lambda x: 2.0*x + 1.0

xs = tuple([random.uniform(-1,1) for _ in range(1000)])    # 1000 points
ys = tuple([f(x)+0.1*random.gauss(0,1) for x in xs])


plt.title("Dataset")
plt.scatter(xs, ys, s=1)
plt.plot(xs, [f(x) for x in xs], label=f"y =  2x+1")
plt.legend()
plt.show()

In [ ]:
lin = LinearModel(0.,0.)
history = [lin.get_params()]

for epoch in range(500):
  grad = d_mse(lin, xs, ys)
  update(lin, 0.01, **grad)
  err = mse(lin, xs, ys)
  params = lin.get_params()
  history.append(params)
  print(f"Epoch {epoch+1}: mse={err:.4f}, w={params.get('weight'):.4f}, b={params.get('bias'):.4f}")

In [ ]:
import plotly.express as px
import pandas as pd

df = pd.DataFrame(history, columns=['weight','bias'])
df = df.set_index(df.index.set_names('epoch')).reset_index()
df0 = df.copy()
df1 = df.copy()
df0['x'] = min(xs)
df1['x'] = max(xs)
df = pd.concat([df0, df1]).reset_index(drop=True)
df['y'] = df.weight * df.x + df.bias

fig = px.line(df, x='x', y='y', animation_frame="epoch", width=500, height=500)

fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 0.1
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 0.1
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['redraw'] = True

fig.add_scatter(x=xs, y=ys, mode='markers', name='data', marker={'size':2})

for i, frame in enumerate(fig.frames):
    frame['layout']['title_text'] = f"Prediction: y = {history[i]['weight']:.4f}x{'' if history[i]['bias'] < 0 else '+'}{history[i]['bias']:.4f}"

fig.update_layout(template='plotly_dark')
fig.show()